In [1]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import norm

import matplotlib.pyplot as plt

In [2]:
downweight_granularity=['dtime','Direction','dday','month']

# Data Import

In [3]:
Data=pd.read_csv('R_Output_Test_Pax.csv',sep=',')
Data=Data[['DepDate','yday','dtime','Direction','month','dday','Dprio','NumPax']]

Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

Data=Data.loc[Data['TicketDate']<=pd.to_datetime('2019-03-27'),:]

Data=Data[[x for x in Data.columns if x!='TicketDate']]

Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6]).sum().groupby(level=[0,1,2,3,4,5])['NumPax'].cumsum().reset_index()

Data['Dprio']=-Data['Dprio']

In [4]:
IdealCurve=pd.read_csv('IdealCurve.csv')

In [5]:
Data=Data.merge(IdealCurve,on=downweight_granularity+['Dprio'])

In [6]:
Data=Data[['DepDate','dtime','Direction','dday','Dprio','NumPax','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]\
.sort_values(by=['DepDate','Direction','dtime','Dprio'])

In [7]:
Data['NumPax']=Data['NumPax'].apply(lambda x: int(round(x)))
Data['Ideal_lower']=Data['Ideal_lower'].apply(lambda x: int(round(x)))
Data['Ideal_average_downweighted']=Data['Ideal_average_downweighted'].apply(lambda x: int(round(x)))
Data['Ideal_average_full']=Data['Ideal_average_full'].apply(lambda x: int(round(x)))
Data['Ideal_upper']=Data['Ideal_upper'].apply(lambda x: int(round(x)))

In [8]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [9]:
Data=Data.sort_values(by=['DepDate','dtime'])
Data=Data.rename(columns={'NumPax': 'Actual Bookings', 'Ideal_lower': 'Ramp-up frontier',
'Ideal_average_downweighted': 'Ideal curve (80% LF)', 'Ideal_average_full': 'Ideal curve (100% LF)',
'Ideal_upper': 'Phase-down frontier'})

In [10]:
Data.to_csv('GUI_in.csv',index=False)

In [11]:
Data.loc[Data['DepDate']==pd.to_datetime('2019-04-13'),:]

,DepDate,dtime,Direction,dday,Dprio,Actual Bookings,Ramp-up frontier,Ideal curve (80% LF),Ideal curve (100% LF),Phase-down frontier
53946,2019-04-13,06:45,O,Saturday,17,36,37,44,55,60
53947,2019-04-13,06:45,O,Saturday,18,36,36,43,54,60
53948,2019-04-13,06:45,O,Saturday,19,36,36,43,53,59
53949,2019-04-13,06:45,O,Saturday,20,36,35,42,53,59
53950,2019-04-13,06:45,O,Saturday,21,36,34,41,52,58
53951,2019-04-13,06:45,O,Saturday,22,36,33,41,51,57
53952,2019-04-13,06:45,O,Saturday,23,34,33,40,50,57
53953,2019-04-13,06:45,O,Saturday,24,34,32,40,50,56
53954,2019-04-13,06:45,O,Saturday,25,33,31,39,49,55
53955,2019-04-13,06:45,O,Saturday,26,33,31,38,48,55


In [12]:
Data

,DepDate,dtime,Direction,dday,Dprio,Actual Bookings,Ramp-up frontier,Ideal curve (80% LF),Ideal curve (100% LF),Phase-down frontier
53245,2019-04-11,06:45,O,Thursday,15,39,32,40,50,56
53246,2019-04-11,06:45,O,Thursday,16,37,30,39,48,55
53247,2019-04-11,06:45,O,Thursday,17,35,29,38,47,54
53248,2019-04-11,06:45,O,Thursday,18,35,28,37,46,53
53249,2019-04-11,06:45,O,Thursday,19,35,27,36,45,52
53250,2019-04-11,06:45,O,Thursday,20,32,26,35,44,51
53251,2019-04-11,06:45,O,Thursday,21,31,25,34,43,50
53252,2019-04-11,06:45,O,Thursday,22,30,24,33,42,49
53253,2019-04-11,06:45,O,Thursday,23,29,23,33,41,48
53254,2019-04-11,06:45,O,Thursday,24,29,22,32,40,48
